In [1]:
import gymnasium as gym
import gym_super_mario_bros

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from nes_py.wrappers import JoypadSpace

In [33]:
# Parallel environments
env = gym_super_mario_bros.make('SuperMarioBros-v0', render_mode='rgb_array', apply_api_compatibility=True)
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
env = JoypadSpace(env, [["right"], ["right", "A"], ['right', 'A', 'B']])
# vec_env = make_vec_env(env, n_envs=4)

model = PPO("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=500000)
model.save('ppo_mario')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
-----------------------------
| time/              |      |
|    fps             | 238  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 160          |
|    iterations           | 2            |
|    time_elapsed         | 25           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0109122265 |
|    clip_fraction        | 0.131        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | -0.00362     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.965        |
|    n_updates            | 10           |
|    poli

In [ ]:
model = PPO.load("ppo_mario")

In [34]:
env = model.get_env()
state = env.reset()
frames = [state]
for step in range(5000):
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    frames.append(state)
    if done:
        break
env.close()

In [35]:
frames = np.concatenate(frames)
frames = np.swapaxes(frames, 1, 3)
frames = np.swapaxes(frames, 1, 2)

In [36]:
import cv2
import numpy as np

def create_video(frames, output_video_path, fps=30):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for frame in frames:
        out.write(frame)

    out.release()

create_video(frames, 'out.mp4')